# 데이터 분석을 위한 추가 전처리
1. 'stos', 'dtos' 제거
2. 'label' 에서 'target 생성 및 label 제거
3. 숫자형 데이터 MinMaxScaling

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [4]:
dataset=pd.read_csv('/content/drive/MyDrive/캡스톤/dataset/CTU-13(10)_preprocessed.csv')

In [5]:
# column 명들 소문자로 변환
dataset.columns = dataset.columns.str.lower()

In [6]:
# 'stos0' 'dtos0' 모두 1.0 의 값을 가지므로 분류에 필요하지 않은 것 같아 삭제 (10번 시나리오에서는)
dataset.drop(['stos0','dtos0'],axis=1,inplace=True)
dataset

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,totpkts,totbytes,srcbytes,label
0,3587.569824,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122194,0.000743,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,196,98,flow=From-Normal-V51-Grill
122195,0.000913,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2,196,98,flow=From-Normal-V51-Grill
122196,0.000414,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,244,81,flow=From-Normal-V51-Stribrek
122197,0.000322,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,2,280,81,flow=From-Normal-V51-Stribrek


In [7]:
# Botnet 과 Normal 구분 -> 정상 0, 비정상 1
def convertLabel(sample):
  if 'Botnet' in sample: return 1
  else: return 0

dataset['target'] = dataset['label'].apply(convertLabel)

In [8]:
# label feature 제거
dataset.drop(['label'],axis=1,inplace=True)

In [9]:
# 숫자형 feature -> MinMaxScaling
columns=['dur','totpkts','totbytes','srcbytes']
scaler=MinMaxScaler()
dataset[columns]=scaler.fit_transform(dataset[columns])

## Train data, Test data 나누기
**논문에 나와있는 대로**
1. Train data -> Normal data(40%)
2. Test data -> Normal data(60%) + Abnormal data(60%)

In [10]:
# 정상 dataset / 비정상 dataset
normal_dataset=dataset[dataset['target']==0]
abnormal_dataset=dataset[dataset['target']==1]

# 정상 dataset -> Train 정상 / Test 정상 나누기 , 40%:60%
normal_dataset = normal_dataset.drop(['target'],axis=1) # 'target' feature drop 
train_normal,test_normal=train_test_split(normal_dataset, test_size=0.6, random_state=42)

In [11]:
# 비정상 dataset에서 random 으로 60% 추출
abnormal_dataset = abnormal_dataset.sample(frac=0.6)
abnormal_dataset = abnormal_dataset.drop(['target'],axis=1) # 'target' feature drop 

# Dataframe 을 pytorch tensor로 변환 

In [12]:
# pytorch tensor 로 변환 

# determine the supported device
def get_device():
  if torch.cuda.is_available():
    device = torch.device('cuda:0')
  else :
    device = torch.device('cpu') # don 't have GPU 
  return device

# convert a df to tensor to be used in pytorch
def dataframe_to_tensor(df):
   device = get_device()
   return torch.from_numpy(df.values).float().to(device)

In [13]:
train_normal = dataframe_to_tensor(train_normal)

# Deep K-means
1. layer 수는 5
2. mini-batch size 는 100 
3. 각 layer에서 얼만큼 줄일지 + 활성화 함수는 어떤 것을 사용할지 ?
4. learning rate=0.001 인데 어느 정도로 해야할지?

In [14]:
train_loader = DataLoader(train_normal,batch_size=100,shuffle=True)
train_batch = iter(train_loader).next()
print(train_batch.shape)

torch.Size([100, 27])


In [15]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(27,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())
          
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,16),
                      nn.ReLU(),
                      nn.Linear(16,27),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)

In [16]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0)

  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign]

In [17]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [18]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist()

  return cluster_acc(actual,predictions)

#모델 학습


In [34]:
latent_size=4
num_clusters=2

In [35]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=0.001)

In [36]:
T1=100
T2=200
lam = 1e-3
ls = 0.05

In [37]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  # 정상 data(label=0)에 대해서만 학습
  running_loss = 0.0
  for batch in train_loader:
    inputs = batch.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    print('[%d] Train loss:%.4f' %(ep,avg_loss) )

[0] Train loss:0.2397
[10] Train loss:0.0292
[20] Train loss:0.0071
[30] Train loss:0.0031
[40] Train loss:0.0023
[50] Train loss:0.0021
[60] Train loss:0.0020
[70] Train loss:0.0019
[80] Train loss:0.0019
[90] Train loss:0.0018
[100] Train loss:0.0018
[110] Train loss:0.0023
[120] Train loss:0.0021
[130] Train loss:0.0020
[140] Train loss:0.0019
[150] Train loss:0.0018
[160] Train loss:0.0017
[170] Train loss:0.0017
[180] Train loss:0.0017
[190] Train loss:0.0017
[200] Train loss:0.0016
[210] Train loss:0.0016
[220] Train loss:0.0008
[230] Train loss:0.0007
[240] Train loss:0.0007
[250] Train loss:0.0007
[260] Train loss:0.0007
[270] Train loss:0.0006
[280] Train loss:0.0006
[290] Train loss:0.0006


In [38]:
kmeans.centroids

Parameter containing:
tensor([[ 2.7903e+00,  5.6486e+00,  1.5954e+00,  2.2993e-41],
        [ 4.1532e-01,  2.5053e+00,  2.1851e+00, -2.8137e-40]], device='cuda:0',
       requires_grad=True)

In [39]:
torch.save(encoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13)/encoder.pth')
torch.save(decoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13)/decoder.pth')
torch.save(kmeans.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13)/kmeans.pth') # kmeans 매개변수 : 클러스터 개수와 중심들